In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.image.random_contrast(image, lower=0.2, upper=2.0)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, lower=0.2, upper = 0.2)
    image = tf.image.random_hue(image, max_delta = 0.08)
    image = tf.image.random_saturation(image, lower=0.2, upper=0.2)
    return image, label

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(4).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [9]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [10]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [11]:
history = model.fit(train_data,
                    epochs=25,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 11:46:02.330075: {'loss': 1.8981878023386, 'accuracy': 0.5562125, 'val_loss': 1.3884707488072145, 'val_accuracy': 0.6239}
Epoch 1 ends at 11:50:38.762453: {'loss': 0.8964221838653088, 'accuracy': 0.73758125, 'val_loss': 0.754434406567531, 'val_accuracy': 0.7523}
Epoch 2 ends at 11:55:06.937367: {'loss': 0.462691828417778, 'accuracy': 0.8450438, 'val_loss': 0.7746104207008507, 'val_accuracy': 0.7621}
Epoch 3 ends at 11:59:38.571821: {'loss': 0.27020333376079797, 'accuracy': 0.90651876, 'val_loss': 0.7700450530477391, 'val_accuracy': 0.788}
Epoch 4 ends at 12:04:14.627420: {'loss': 0.1721088802497834, 'accuracy': 0.940025, 'val_loss': 0.963523471241544, 'val_accuracy': 0.7613}
Epoch 5 ends at 12:09:06.069114: {'loss': 0.13144642582051455, 'accuracy': 0.95489377, 'val_loss': 1.1291822033702947, 'val_accuracy': 0.7472}
Epoch 6 ends at 12:13:44.335537: {'loss': 0.10832988434750587, 'accuracy': 0.96300626, 'val_loss': 0.80517285673102, 'val_accuracy': 0.8194}
Epoch 7 ends at 

In [12]:
model.evaluate(test_data)

157/157 [==============================].3289 - accuracy: 0.82 - 0s 66ms/step - loss: 1.6641 - accuracy: 0.82 - 0s 60ms/step - loss: 1.6199 - accuracy: 0.82 - 0s 57ms/step - loss: 1.3384 - accuracy: 0.84 - 0s 54ms/step - loss: 1.1963 - accuracy: 0.86 - 0s 53ms/step - loss: 1.1213 - accuracy: 0.86 - 0s 53ms/step - loss: 1.0699 - accuracy: 0.86 - 0s 52ms/step - loss: 1.2086 - accuracy: 0.85 - 0s 51ms/step - loss: 1.1472 - accuracy: 0.85 - 0s 50ms/step - loss: 1.1093 - accuracy: 0.85 - 1s 49ms/step - loss: 1.0630 - accuracy: 0.85 - 1s 48ms/step - loss: 1.0787 - accuracy: 0.85 - 1s 47ms/step - loss: 1.0853 - accuracy: 0.84 - 1s 47ms/step - loss: 1.0805 - accuracy: 0.84 - 1s 47ms/step - loss: 1.0723 - accuracy: 0.84 - 1s 47ms/step - loss: 1.1262 - accuracy: 0.84 - 1s 47ms/step - loss: 1.1062 - accuracy: 0.84 - 1s 47ms/step - loss: 1.1230 - accuracy: 0.84 - 1s 46ms/step - loss: 1.1066 - accuracy: 0.84 - 1s 46ms/step - loss: 1.1127 - accuracy: 0.84 - 1s 46ms/step - loss: 1.0721 - accuracy: 0.

[1.1554142274674337, 0.8252]

In [13]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))